# Top tokens for different categories

## Read mapping information

In [1]:
import os

import pandas as pd


demoTrainDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240614/data/wb_365_wa_1/splits_v1/demographics/mortality_normal_train.csv', sep='\t')
demoTestDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240614/data/wb_365_wa_1/splits_v1/demographics/mortality_normal_test.csv', sep='\t')
demoValidateDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240614/data/wb_365_wa_1/splits_v1/demographics/mortality_normal_validate.csv', sep='\t')
demoDf = pd.concat([demoTrainDf, demoTestDf, demoValidateDf], ignore_index=True)
demoDf

,person_id,visit_occurrence_id,TYPE,COUNTRY_OF_BIRTH,SEX,Age_Bin,JOURNEY_ID
0,678487,3312694,Emergency,Australia,Male,"(60, 70]",3312694
1,2094045,9199249,Emergency,Australia,Male,"(20, 30]",9199249
2,2107813,9410987,Emergency,Australia,Male,"(50, 60]",9410987
3,2083212,602890,Inpatient,Australia,Female,"(40, 50]",9039437
4,2083212,9039437,Emergency,Australia,Female,"(40, 50]",9039437
...,...,...,...,...,...,...,...
7194,2603535,16588204,Inpatient,Australia,Male,"(60, 70]",16588204
7195,2603535,16709341,Inpatient,Australia,Male,"(60, 70]",16709341
7196,2603535,16651755,Inpatient,Australia,Male,"(60, 70]",16651755
7197,2098616,16872774,Emergency,Australia,Female,"(80, 90]",16872774


In [2]:
import os

mappingDf = pd.read_csv(os.environ['GENOMICS_DATA_BASE'] + '/patient_tube_id_mapping_full.tsv', sep='\t')
mappingDf

,tube_code,PATIENT_ID,db_ID,pt_age,pt_gender,EPISODE_ID,hospital_admission,hospital_discharge,hospital_in_last_year,genome_species,species_reported,contig_number,length,tube_code_duplicate_or_old,date_of_collection,ID_number,location_additional_02,collected_from_original
0,AH19J072,213972.0,3JNQS,64,Male,12931474.0,2019-10-17,2019-11-29,yes,Escherichia coli,Escherichia coli,92,5131021,NaN,2019-10-30,19-303-0997,A-7EA;HAEM,Blood
1,AH19J074,213972.0,3JNQS,64,Male,12931474.0,2019-10-17,2019-11-29,yes,Streptococcus sp. D19,Streptococcus mitis group,84,2022991,NaN,2019-10-30,19-303-1174,A-7EA;HAEM,Blood Peripheral
2,AH19B003,526238.0,567AE,85,Male,640261.0,2019-02-04,2019-02-07,no,Escherichia coli,Escherichia coli,93,4942915,NaN,2019-02-03,19-034-1596,A-4WB;GMC,Blood
3,AH20A024,788941.0,3YBDS,58,Female,13194824.0,2020-01-16,2020-01-19,no,Escherichia coli,Escherichia coli,110,5149157,NaN,2020-01-16,20-016-2112,S-ED;Emergency,Blood Venous
4,AH20L041,1023735.0,2VJ78,42,Male,14210147.0,2020-12-18,2021-01-05,yes,Staphylococcus aureus,Staphylococcus aureus,46,2735216,NaN,2020-12-16,20-351-2856,Emergency Dept;nan,Blood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3112,ALF22L076,639020.0,3S6HL,84,Male,17219899.0,2022-12-15,2022-12-20,yes,Streptococcus vestibularis,Streptococcus salivarius group,107,1938627,NaN,2022-12-15,22-349-2150,Ward 4WB,Blood Peripheral
3113,ALF22L089,655404.0,6EWBM,67,Female,NaN,NaN,NaN,NaN,Proteus columbae,Proteus vulgaris,152,3972219,NaN,2022-12-17,22-351-1465,Comm/Amb Clinic,Blood
3114,ALF22L138,2572048.0,YM5PJ,61,Male,NaN,NaN,NaN,NaN,Enterobacter roggenkampii,Enterobacter cloacae complex,66,4876179,NaN,2022-12-27,22-361-0202,Ward 7 East,Hick White Lum
3115,ALF23A102,2054964.0,5DYAH,35,Female,17294287.0,2023-01-12,2023-02-01,yes,Streptococcus constellatus,Streptococcus anginosus group,54,1869828,NaN,2023-01-11,23-011-3305,ICU,Blood Venous


In [3]:
mappedJourneyTubeIdsDf = mappingDf[['tube_code', 'PATIENT_ID', 'EPISODE_ID']].drop_duplicates().merge(
    demoDf[['person_id', 'visit_occurrence_id', 'JOURNEY_ID']],
    how='inner',
    left_on=['PATIENT_ID', 'EPISODE_ID'],
    right_on=['person_id', 'visit_occurrence_id']
).drop(
    columns=['PATIENT_ID', 'EPISODE_ID', 'visit_occurrence_id']
)[['person_id', 'JOURNEY_ID', 'tube_code']].drop_duplicates()
mappedJourneyTubeIdsDf

,person_id,JOURNEY_ID,tube_code
0,50056,13224842,AH20B011
1,62567,12645629,AH19G065
2,138108,14737781,AH21E052
3,211314,13955099,AH20J009
4,211314,13955099,KPN2214
...,...,...,...
512,2084373,16158768,ALF22C004
513,1058066,16759482,ALF22H089
514,2552781,16966609,ALF22J054
515,2638484,16969999,ALF22K143


## Read bed files containing overlapping tokens and attribution scores

In [4]:
import os
import pandas as pd
from pathlib import Path


overlappingFilesDir = Path(os.environ['GENOMICS_DATA_BASE'], 'genome_nlp_tokens', 'overlapping_with_annotations')

overlappingDfList = []
for overlappingFile in os.listdir(overlappingFilesDir):
    df = pd.read_csv(Path(overlappingFilesDir, overlappingFile), sep='\t', names=['contig_id', 'start_position', 'end_position', 'tokens', 'score', 'id', 'name', 'gene', 'atributes'])
    df['tube_code'] = [overlappingFile.split('_')[0]]*df.shape[0]
    overlappingDfList.append(df)
overlappingDf = pd.concat(overlappingDfList, ignore_index=True)
overlappingDf = overlappingDf.merge(
    mappedJourneyTubeIdsDf,
    how='inner',
    on=['tube_code']
)
overlappingDf = overlappingDf[['tube_code', 'contig_id', 'start_position', 'end_position', 'tokens', 'score']].drop_duplicates()
overlappingDf

,tube_code,contig_id,start_position,end_position,tokens,score
0,AH19E065,1,191,192,GT,-0.001426
1,AH19E065,1,193,197,ATAAC,-0.005218
2,AH19E065,1,198,199,GC,-0.000682
3,AH19E065,1,200,203,GCCA,-0.010548
4,AH19E065,1,204,210,ATTGATT,-0.004120
...,...,...,...,...,...,...
23487,AH19C091,18,149,152,GCTC,0.012179
23488,AH19C091,18,153,158,TAACCA,0.012327
23489,AH19C091,18,159,164,GCTGAG,-0.020735
23490,AH19C091,18,165,170,CTATAG,-0.013398


## Top 20 tokens (Overall)

In [8]:
overlappingDf.tokens.value_counts().reset_index().sort_values(by=['count'], ascending=False)[:20]

,tokens,count
0,CG,212
1,TTC,116
2,CGT,107
3,CGG,98
4,ACG,77
5,GC,75
6,ATGAT,61
7,GCAAT,60
9,CGAC,58
8,AAGTT,58


## Top 20 tokens with high attribution scores (score > 0.02)

In [9]:
overlappingDf[overlappingDf.score > 0.02].tokens.value_counts().reset_index().sort_values(by=['count'], ascending=False)[:20]

,tokens,count
0,GCTC,8
1,ATCC,7
2,CGT,7
3,GTGC,6
4,CTCAGC,6
5,CAC,6
6,AGAGC,6
7,ATCT,6
8,ATTGAG,6
9,CCGG,6


## Top 20 tokens without considering the high attribution scores (score < 0.02)

In [12]:
overlappingDf[overlappingDf.score < 0.02].tokens.value_counts().reset_index().sort_values(by=['count'], ascending=False)[:20]

,tokens,count
0,CG,207
1,TTC,116
2,CGT,100
3,CGG,98
4,ACG,77
5,GC,75
6,ATGAT,61
7,GCAAT,60
8,CGAC,58
9,AAGTT,57


## Top 20 tokens with low attribution scores (score < -0.02)

In [11]:
overlappingDf[overlappingDf.score < -0.02].tokens.value_counts().reset_index().sort_values(by=['count'], ascending=False)[:20]

,tokens,count
0,CG,17
1,CGAA,15
3,GC,14
2,CCC,14
4,CGT,13
5,GACCCC,12
6,GAACCC,12
8,TTC,12
7,CCTGC,12
9,GACC,11


## Top 20 tokens without considering the low attribution scores (score > -0.02)

In [13]:
overlappingDf[overlappingDf.score > -0.02].tokens.value_counts().reset_index().sort_values(by=['count'], ascending=False)[:20]

,tokens,count
0,CG,195
1,TTC,104
2,CGT,94
3,CGG,93
4,ACG,77
5,GC,61
7,AAGTT,58
6,ATGAT,58
8,GCAAT,58
9,CCG,56
